In [ ]:
import os
import numpy as np
import tensorflow as tf

from ml.config.train import default_cfg
from ml.config.train import cfg as cfg_train
from ml.config.test import cfg as cfg_test
from ml.dataset import odgt2train, odgt2test
from ml.models import TASK_MODEL_DICT, ModelBuilder, OptimizerBuilder, BestModelCallback

from sklearn.metrics import classification_report

print("All packages imported!")

## Training

### Config

In [ ]:
task = 1
part = 'dynamic'

# fill in your directory set up here
config_fp = f'/home/jonathan/git/pdiot-ml/config/train/task_{task}/{part}.yaml'
cfg_train.merge_from_file(config_fp)
cfg_train.DATASET.path = '/home/jonathan/git/pdiot-ml/data/sets/pdiot-data/'
cfg_train.DATASET.LIST.train = f'train_{part}_pdiot-data.odgt' 
cfg_train.DATASET.LIST.val = f'val_{part}_pdiot-data.odgt'
print(cfg_train)

### Data

In [ ]:
train_odgt_fp = os.path.join(cfg_train.DATASET.path, cfg_train.DATASET.LIST.train)
val_odgt_fp = os.path.join(cfg_train.DATASET.path, cfg_train.DATASET.LIST.val)

train_X, train_y = odgt2train(train_odgt_fp, cfg_train.MODEL.INPUT.window_size, 
                                            cfg_train.TRAIN.DATA.overlap_size)
val_X, val_y = odgt2train(val_odgt_fp, cfg_train.MODEL.INPUT.window_size, 
                                      cfg_train.TRAIN.DATA.overlap_size)

### Optimization

In [ ]:
model = ModelBuilder.build_classifier(cfg_train.MODEL, '', cfg_train.DATASET.num_classes)
optimizer = OptimizerBuilder.build_optimizer(cfg_train.TRAIN.OPTIM)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = [
    tf.keras.metrics.SparseCategoricalAccuracy(name='acc'),
]
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=cfg_train.TRAIN.LEN.early_stop)
best_model_callback = BestModelCallback(model)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
history = model.fit(train_X, train_y, 
                    validation_data=(val_X, val_y), 
                    epochs=cfg_train.TRAIN.LEN.num_epoch, 
                    batch_size=cfg_train.TRAIN.DATA.batch_size,
                    callbacks=[early_stop_callback, best_model_callback])

### Evaluation

In [ ]:
val_pred = np.argmax(model(val_X), axis=1)
print(classification_report(val_pred, val_y))

## Testing

### Config

In [ ]:
task = 1
subject = 0

# fill in your directory set up here
project_dp = '/home/jonathan/git/pdiot-ml/'
config_fp = os.path.join(project_dp, f'config/test/task_{task}.yaml')
cfg_test.merge_from_file(config_fp)
cfg_test.TEST.subject = subject
cfg_test.DATASET.path = '/home/jonathan/git/pdiot-ml/data/sets/pdiot-data/'
cfg_test.TEST.path = f'/home/jonathan/git/pdiot-ml/ckpt/test/task_{task}'   
print(cfg_test)

### Data

In [ ]:
test_odgt_fp = os.path.join(cfg_test.DATASET.path, cfg_test.DATASET.odgt.format(cfg_test.DATASET.task))
test_dict = odgt2test(test_odgt_fp, cfg_test.DATASET.task,
                                    cfg_test.TEST.subject, 
                                    cfg_test.MODEL.INPUT.window_size, 
                                    cfg_test.TEST.DATA.overlap_size)

### Optimization

In [ ]:
components = TASK_MODEL_DICT[cfg_test.DATASET.task]
model_dict = {}
for component in components:
    print(component)
    component_cfg_fp = os.path.join(project_dp, cfg_test.MODEL.CONFIG[component])
    cfg_train = default_cfg()
    cfg_train.merge_from_file(component_cfg_fp)

    model = ModelBuilder.build_classifier(cfg_train.MODEL, '', cfg_train.DATASET.num_classes)
    optimizer = OptimizerBuilder.build_optimizer(cfg_train.TRAIN.OPTIM)
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    metrics = [
        tf.keras.metrics.SparseCategoricalAccuracy(name='acc'),
    ]

    early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=cfg_train.TRAIN.LEN.early_stop)

    valid_idx = test_dict['train'][component] != -1
    train_X = test_dict['train']['X'][valid_idx]
    train_y = test_dict['train'][component][valid_idx]

    valid_idx = test_dict['val'][component] != -1
    val_X = test_dict['val']['X'][valid_idx]
    val_y = test_dict['val'][component][valid_idx]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    history = model.fit(train_X, train_y, 
                        validation_data=(val_X, val_y),
                        epochs=cfg_train.TRAIN.LEN.num_epoch, 
                        batch_size=cfg_test.TEST.DATA.batch_size,
                        callbacks=[early_stop_callback])
    
    model_dict[component] = model

### Evaluation

In [ ]:
model = ModelBuilder.build_hierarchical_classifier(cfg_test.DATASET.task, model_dict)

pred = model(test_dict['val']['X'])

report = classification_report(test_dict['val']['y'], pred)

print(f'Task {cfg_test.DATASET.task} - Subject {cfg_test.TEST.subject}') 
print(report)